## Формирование трейнового датасета

Для обучения модели мы будем использовать данные, которые при помощи парсинга получили самостоятельно с сайта auto.ru, 
а также датасет, который был собран в рамках соревнования ранее (08.11.2020, далее - "ноябрьский датасет") и выложен на Каггл.

В данном файле пример обработки собранных данных. Данные, полученные позднее, обрабатывались таким же образом.

Свои данные мы приведем к формату ноябрьского датасета.

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 550) # больше колонок

In [4]:
# Результаты парсинга всех машин по России от 22.02.21
latest_df = pd.read_csv('allCars22.02.2021.csv')

# Результаты парсинга машин по Москве и МО от 23.02.21
latest_moscow_df = pd.read_csv('all_moscow+200km.csv')

# Данные с Каггла - парсинг по Москве и области от 08.11.20
eda_df = pd.read_csv('car_price_08_11_2020_final.csv')

# Тестовый датасет
test = pd.read_csv('test.csv')

### Сначала разбираемся с результатами своего парсинга

In [411]:
def data_cleaning(df):
    """Функция первичной очистки данных после парсинга"""
    
    data = df.copy()
    # Удаляем ненужные колонки
    data.drop('Unnamed: 0', axis = 1, inplace=True)

    # В данные попадают заголовки для csv-таблицы. Удалим их
    data = data.loc[data.brand != 'brand']

    # Удалим дубли по признаку sell_id
    data.drop_duplicates(subset=['sell_id'], keep='last', inplace=True)
    
    return data


def id_set(df):
    """ Функция возвращает уникальные id объявлений в датасете"""
      
    return set(df['sell_id'].tolist())

In [412]:
# Исходные размеры датасетов
print(f"Size of latest_df: {latest_df.shape}")
print(f"Size of latest_moscow_df: {latest_moscow_df.shape}")

Size of latest_df: (170306, 35)
Size of latest_moscow_df: (50496, 35)


In [413]:
# список уникальных id объявлений для проверки совпадений в датасетах
latest_id = id_set(latest_df)
latest_moscow_id = id_set(latest_moscow_df)

In [414]:
# Количество пересечений. За один день в Москве подали 11,5 тыс объявлений??? Странно...
intersect = latest_id & latest_moscow_id
len(intersect)

37556

In [415]:
# Удалим пересечения с Москвой
latest_df = latest_df[latest_df['sell_id'].isin(intersect) == False]
print(f"Size of latest_df: {latest_df.shape}. {len(intersect)} intersections deleted")

Size of latest_df: (130424, 35)


In [416]:
# Добавляем локацию, тк мы собирали данные по всей России (0), а не только по Москве и МО (1)
latest_df['moscow'] = 0
latest_moscow_df['moscow'] = 1
eda_df['moscow'] = 1

In [417]:
# Объединяем данные парсинга
parsed_data = pd.concat([latest_df, latest_moscow_df], ignore_index=True)
print(f"Size of parsed_data: {parsed_data.shape}")

Size of parsed_data: (180920, 36)


In [418]:
parsed_data = data_cleaning(parsed_data)
print(f"Size of parsed_data: {parsed_data.shape}")

Size of parsed_data: (176684, 35)


#### Преобразуем признаки для соответствия ноябрьскому датасету

In [420]:
# Словари для преобразования признаков

fuel_type_converter = {
    'GASOLINE': 'бензин',
    'DIESEL': 'дизель',
    'HYBRID': 'гибрид',
    'ELECTRO': 'электро',
    'LPG': 'газ',
}

color_convertor = {
    '040001': 'черный',
    'FAFBFB': 'белый',
    '0000CC': 'синий',
    '97948F': 'серый',
    'CACECB': 'серебристый',
    '200204': 'коричневый',
    'EE1D19': 'красный',
    '007F00': 'зелёный',
    '22A0F8': 'голубой',
    'C49648': 'бежевый.',
    'DEA522': 'золотистый',
    'FF8649': 'оранжевый',
    '660099': 'фиолетовый',
    '4A2197': 'пурпурный',
    'FFD600': 'жёлтый',
    'FFC0CB': 'розовый',    
}

wheel_convertor = {
    'LEFT': 'Левый',
    'RIGHT': 'Правый',
}

transmission_converter = {
    'MECHANICAL': 'механическая',
    'AUTOMATIC': 'автоматическая',
    'ROBOT': 'роботизированная',
    'VARIATOR': 'вариаторная'
}

gear_type_convertor = {
    'REAR_DRIVE': 'задний',
    'ALL_WHEEL_DRIVE': 'полный',
    'FORWARD_CONTROL': 'передний',
}

pts_convertor = {
    'ORIGINAL': 'Оригинал',
    'DUPLICATE': 'Дубликат',
}

In [421]:
# Несколько строчек съехали. Удалим их
colors = list(color_convertor.keys())
parsed_data = parsed_data[parsed_data['color_hex'].isin(colors)]
print(f"Size of parsed_data: {parsed_data.shape}")

Size of parsed_data: (176677, 35)


Ниже та же обработка, которая была сделана для ноябрьского датасета

In [422]:
def convert_features(df):
    """ Функция преобразования признаков к виду тестового датасета"""
    data = df.copy()
    
    data.fuelType.replace(fuel_type_converter, inplace=True)

    # преобразуем значения цветов из hex-формата в строчное название цвета
    data['color'] = data.color_hex.apply(lambda x: color_convertor[x])
    data.drop(labels='color_hex', axis=1, inplace=True)
    
    # Преобразуем признак steering_wheel
    data['Руль'].replace(wheel_convertor, inplace=True)
    
    # Преобразуем признак vehicleTransmission
    data.vehicleTransmission.replace(transmission_converter, inplace=True)
    
    # Преобразуем признак gear_type
    data['Привод'].replace(gear_type_convertor, inplace=True)
    
    # Преобразуем признак ПТС
    data['ПТС'].replace(pts_convertor, inplace=True)
    
      
    return data

In [423]:
parsed_data = convert_features(parsed_data)

In [424]:
# удалим объекты без цены и с пропусками в след. признаках
# удаление нужно в т.ч.для того, чтобы перевести данные в int(Nan в int не переводятся)
cols_to_dropna = 'bodyType engineDisplacement enginePower fuelType modelDate name numberOfDoors super_gen vehicleConfiguration vehicleTransmission Привод price price_segment'.split(' ')

parsed_data = parsed_data.dropna(subset=cols_to_dropna, axis=0)
print(f"Size of parsed_data: {parsed_data.shape}")


Size of parsed_data: (176000, 35)


In [425]:
# Почти все данные получились типа 'object'. В дальнейшем это надо исправить 
parsed_data.dtypes.value_counts()

object    34
int64      1
dtype: int64

In [426]:
# Посмотрим, какие признаки в ноябрьском датасете числовые
int_types = eda_df.dtypes[eda_df.dtypes == 'int64'].index.to_list()
float_types = eda_df.dtypes[eda_df.dtypes == 'float64'].index.to_list()

In [427]:
# Приводим в соответствие
parsed_data[int_types] = parsed_data[int_types].astype('int64')
parsed_data[float_types] = parsed_data[float_types].astype('float64')

На всякий случай сохраняем промежуточный результат

In [428]:
def write_to_csv(data, file_name):
    data.to_csv(f"{file_name}.csv", index=False, encoding='utf-8')

In [429]:
write_to_csv(parsed_data, 'parsed_data_22_02_2021')

### Проверяем датасет с Каггла

In [430]:
print(f"Size of eda_df: {eda_df.shape}")

Size of eda_df: (57708, 36)


In [431]:
# Cоздадим список уникальных id датасетов. На этот раз все id принадлежат типу int, можно сравнивать.
# Сравнивать их ранее было бы некорректно.
parsed_data_id = id_set(parsed_data)
eda_id = id_set(eda_df)
test_id = id_set(test)

In [432]:
# Проверим пересечения ноябрьского датасета с нашим датасетом и удалим их
intersect = parsed_data_id & eda_id
eda_df = eda_df[eda_df['sell_id'].isin(intersect) == False]
print(f"Size of eda_df: {eda_df.shape}. {len(intersect)} intersections deleted")

Size of eda_df: (49817, 36). 7891 intersections deleted


In [433]:
# Соединяем в общий датасет
full_data = parsed_data.append(eda_df, sort=False).reset_index(drop=True)
print(f"Size of full_data: {full_data.shape}")

Size of full_data: (225817, 36)


In [434]:
# Проверяем. Получилось - все sell_id уникальны
full_data.sell_id.nunique()

225817

In [435]:
# проверим пересечения объединенного датасета с тестом и удалим их. 22583 пересечения!
full_data_id = id_set(full_data)
intersect = test_id & full_data_id
full_data = full_data[full_data['sell_id'].isin(intersect) == False]
print(f"Size of full_data: {full_data.shape}. {len(intersect)} intersections deleted")

Size of full_data: (203234, 36). 22583 intersections deleted


In [436]:
full_data.sell_id.nunique()

203234

## Сохраняем результат. Это и будет наш трейновый датасет.

Трейновый датасет не содержит пересечений с тестом. 

В трейновом датасете есть колонка "moscow", которой нет в тестовом датасете (1 - Москва и МО, 2 - вся остальная территория России)

In [437]:
write_to_csv(full_data, 'train_22_02_2021')